# Summarizing A Meeting With Key Points & Tasks From Audio Transcribed by OpenAI Whisper

&nbsp;

This is a Colab notebook that allows you to upload audio files to [OpenAI's free Whisper speech recognition model](https://openai.com/blog/whisper/).

To use it, choose `Runtime->Run All` from the Colab menu. To make it run faster, make sure to change the `Change the runtime type` and select `v100` GPU as the model uses GPUs to speed up the transcription.

You can also upload your own audio samples using the folder icon on the left of this page. That gives you access to a file system you can upload to by dragging files into it. You can see examples of how to run the transcription in a couple of the cells below.

## Install the Whisper Code

In [ ]:
%pip install git+https://github.com/openai/whisper.git -q
%pip install python-docx openai python-dotenv

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 17.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Create Summarization Functions

In [ ]:
import openai
from docx import Document

def meeting_minutes(transcription):
    abstract_summary = abstract_summary_extraction(transcription)
    key_points = key_points_extraction(transcription)
    action_items = action_item_extraction(transcription)
    sentiment = sentiment_analysis(transcription)
    return {
        'abstract_summary': abstract_summary,
        'key_points': key_points,
        'action_items': action_items,
        'sentiment': sentiment
    }

def abstract_summary_extraction(transcription):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are a highly skilled AI trained in language comprehension and summarization. I would like you to read the following text and summarize it into a concise abstract paragraph. Aim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text. Please avoid unnecessary details or tangential points."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    return response['choices'][0]['message']['content']


def key_points_extraction(transcription):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are a proficient AI with a specialty in distilling information into key points. Based on the following text, identify and list the main points that were discussed or brought up. These should be the most important ideas, findings, or topics that are crucial to the essence of the discussion. Your goal is to provide a list that someone could read to quickly understand what was talked about."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    return response['choices'][0]['message']['content']


def action_item_extraction(transcription):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are an AI expert in analyzing conversations and extracting action items. Please review the text and identify any tasks, assignments, or actions that were agreed upon or mentioned as needing to be done. These could be tasks assigned to specific individuals, or general actions that the group has decided to take. Please list these action items clearly and concisely."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    return response['choices'][0]['message']['content']

def sentiment_analysis(transcription):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "As an AI with expertise in language and emotion analysis, your task is to analyze the sentiment of the following text. Please consider the overall tone of the discussion, the emotion conveyed by the language used, and the context in which words and phrases are used. Indicate whether the sentiment is generally positive, negative, or neutral, and provide brief explanations for your analysis where possible."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    return response['choices'][0]['message']['content']

def save_as_docx(minutes, filename):
    doc = Document()
    for key, value in minutes.items():
        # Replace underscores with spaces and capitalize each word for the heading
        heading = ' '.join(word.capitalize() for word in key.split('_'))
        doc.add_heading(heading, level=1)
        doc.add_paragraph(value)
        # Add a line break between sections
        doc.add_paragraph()
    doc.save(filename)



## Upload Audio File For Transcription & Set Variables

In [ ]:
import os
from google.colab import files, drive
from dotenv import load_dotenv

#Mount Google Drive
drive.mount('/content/gdrive', force_remount=True)

#Load .env file and assign OpenAI API Key to variable
_ = load_dotenv("/content/gdrive/MyDrive/Research/apikeys/.env")

openai.api_key = os.environ['OPENAI_API_KEY']

#Upload Audio file
uploaded = files.upload()

for fn in uploaded.keys():

  # Get original filename
  FILENAME = os.path.splitext(fn)[0]

  # Modify filename
  new_filename = FILENAME.replace(' ', '_')
  new_fn = new_filename + os.path.splitext(fn)[1]

  # Get original and new full paths
  original_path = '/content/' + fn
  new_path = '/content/' + new_fn

  # Rename uploaded file
  os.rename(original_path, new_path)

  # Updated variables
  AUDIO_PATH = new_path
  TXT_FILE = new_filename + '.txt'
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Whisper Settings
WHISPER_MODEL = 'large-v2' # tiny.en,tiny,base.en,base,small.en,small,medium.en,medium,large-v1,large-v2,large

# Whisper Variables
WHISPER_DIR = f'/content/{new_filename}'
WHISPER_ZIP_PATH = f'{WHISPER_DIR}.zip'

# Confirm Variable
print('File path: ', AUDIO_PATH)
print('File name: ', new_fn)
print('File name: ', TXT_FILE)
print(f'Renamed {original_path} to {new_path}')
(AUDIO_PATH, WHISPER_MODEL, WHISPER_DIR, WHISPER_ZIP_PATH)

## Transcribe & Summarize

In [ ]:
!whisper --device cuda --model {WHISPER_MODEL} --output_dir {WHISPER_DIR} {AUDIO_PATH}
!cd {WHISPER_DIR} && zip -qr9 "{WHISPER_ZIP_PATH}" *

In [ ]:
transcription = f'{WHISPER_DIR}/{TXT_FILE}'
print(transcription)

In [ ]:
with open(transcription, 'r') as f:
  text = f.read()

minutes = meeting_minutes(text)
print(minutes)

save_as_docx(minutes, new_filename + '_meeting_minutes.docx')

## Download

In [ ]:
files.download(WHISPER_ZIP_PATH)
files.download('/content/' + new_filename + '_meeting_minutes.docx')